<a href="https://colab.research.google.com/github/vktiwary007/stock-move-eureka/blob/main/stockMovement1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [62]:
import numpy as np
import pandas as pd

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

import matplotlib.pyplot as plt

print("Version: ", tf.__version__)
print("Eager mode: ", tf.executing_eagerly())
print("Hub version: ", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices('GPU') else "NOT AVAILABLE")

Version:  2.4.1
Eager mode:  True
Hub version:  0.11.0
GPU is NOT AVAILABLE


In [63]:
import io
df = pd.read_csv('Apple240.csv', skiprows=0)

In [64]:
df[:10]

,Source,News,Movement
0,seekingalpha.com,Bearish Calls Lumentum (NASDAQ: LITE ): It's a...,1
1,seekingalpha.com,The areas to be concerned are that there are f...,1
2,seekingalpha.com,Amazon will soon resume selling the Apple TV (...,1
3,seekingalpha.com,"Facing questions from tech analysts, Apple (NA...",0
4,www.thestreet.com,Cramer and the AAP team say investors need to ...,0
5,seekingalpha.com,One day after Apple acknowledged that it has b...,1
6,www.thestreet.com,1 . -- U.S. stock futures turned lower on Tues...,1
7,seekingalpha.com,"Meanwhile, eight more lawsuits have been filed...",1
8,seekingalpha.com,Shares in Apple and several of its suppliers f...,1
9,www.thestreet.com,1 . -- U.S. stock futures traded slightly high...,0


In [76]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
news = []
print(df.head(5))
stopSet = set(stopwords.words('english'))
for i in range (0, len(df)):
  text = re.sub('[^a-zA-Z]', ' ', df['News'][i])
  text = text.lower()
  text = text.split()
  ps = PorterStemmer()
  text = [ps.stem(word) for word in text if not word in stopSet]
  text = ' '.join(text)
  news.append(text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
              Source  ... Movement
0   seekingalpha.com  ...        1
1   seekingalpha.com  ...        1
2   seekingalpha.com  ...        1
3   seekingalpha.com  ...        0
4  www.thestreet.com  ...        0

[5 rows x 3 columns]


In [77]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(news, df.iloc[:, -1].values, test_size = 0.2, random_state = 0)

train_examples, train_labels = np.array(X_train), np.array(y_train)
test_examples, test_labels = np.array(X_test), np.array(y_test)


In [78]:
train_examples[:10]

array(['q sourc refinitiv ibe consensu estim summari conclus p earn growth expect slow calendar number tell investor quit time blog discuss think reason behind market downdraft start octob see china grow issu growth slow come home roost us earn point alreadi compani give idea next year guidanc start rd quarter earn report everi year real hard guidanc occur januari februari confer call contain th quarter result averag result tech sector quarter expect averag ep growth revenu growth inlin p expect averag ep growth revenu growth lap corpor incom tax rate reduct comp tougher p show slower growth known month technolog look expect slower growth well inlin p howev much depend appl nasdaq aapl result next week fiscal q result tech finish quarter month growth cycl come worri inevit rotat growth momentum like depress tech multipl set reader opportun overweight tech client tech weight reduc summer roughli account weight vari littl earli look better creation commun sector technolog market cap weig

In [79]:
train_labels[:10]

array([0, 1, 0, 0, 0, 1, 0, 0, 0, 1])

In [80]:
print("Training entries: {}, test entries: {}".format(len(train_examples), len(test_examples)))

Training entries: 29251, test entries: 7313


In [81]:
model = "https://tfhub.dev/google/nnlm-en-dim128/2"
hub_layer = hub.KerasLayer(model, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples[:3])

<tf.Tensor: shape=(3, 128), dtype=float32, numpy=
array([[ 9.56805944e-02,  2.25918502e-01,  3.47774744e-01,
        -7.58143723e-01, -7.82843903e-02,  2.89071053e-01,
         3.30699027e-01,  1.85379591e-02,  1.01473063e-01,
         3.40128779e-01,  4.16684866e-01, -2.86378533e-01,
         2.18599647e-01,  4.93880846e-02, -3.66776198e-01,
        -1.01203315e-01,  2.97523081e-01,  2.09660545e-01,
        -3.48484993e-01, -4.25969623e-02,  2.95357592e-03,
        -3.50697100e-01,  2.63485849e-01,  3.45899910e-01,
         2.11549550e-01, -2.53715932e-01,  3.87879699e-01,
         3.31888169e-01, -6.06049657e-01, -6.09188341e-04,
         2.15045482e-01,  1.12114958e-01, -2.15652436e-01,
         3.83189946e-01, -1.46914786e-03, -1.99702218e-01,
        -4.05131802e-02,  3.11892569e-01,  2.20485334e-03,
         2.75814623e-01, -5.55936515e-01, -2.52874613e-01,
        -4.18410331e-01,  1.92072317e-01,  3.21309775e-01,
         8.58628377e-02, -3.65536541e-01,  6.47640824e-01,
      

In [82]:
from sklearn.tree import DecisionTreeClassifier

classifier = DecisionTreeClassifier()
#classifier.fit(train_examples, train_labels)

In [17]:
x_val = train_examples[:10000]
partial_x_train = train_examples[10000:]

y_val = train_labels[:10000]
partial_y_train = train_labels[10000:]

In [83]:
classifier.fit(hub_layer(train_examples), y_train)

DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')

In [84]:
test_val = np.array(hub_layer(test_examples))

print(test_val[:10])

[[-0.17415719  0.15648328  0.07058448 ...  0.2796734   0.2078332
  -0.02255717]
 [ 0.0087497   0.41769493 -0.07398774 ...  0.23198783  0.27141196
  -0.12103764]
 [-0.04271145  0.08316807  0.04242535 ...  0.04279348  0.11769156
  -0.03368118]
 ...
 [-0.08798662  0.10283965  0.32197246 ...  0.47170416  0.31466568
  -0.48935458]
 [-0.32516715  0.25520375 -0.01478244 ...  0.2657958   0.46686307
  -0.35476118]
 [-0.12329755  0.12927084  0.13055408 ...  0.06961498  0.2779784
  -0.15353246]]


In [85]:
y_pred = classifier.predict(test_val)

In [86]:
from sklearn.metrics import accuracy_score, confusion_matrix
print(accuracy_score(test_labels, y_pred)*100,"%")
print(confusion_matrix(test_labels, y_pred))

55.58594284151511 %
[[2450 1501]
 [1747 1615]]
